# Data Science Immersive - General Assembly 
## Capstone Project
Harsha Goonewardana 
Today

### This notebook will further continue feature engineering as a preprocessing step for complex algorithems 

#### Batch Import of dependencies and data 

In [1]:
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
sns.set(style="ticks")
seed=42
import pickle

In [3]:
df =pd.read_pickle('./data/airbnbNYCclean.pkl')

In [4]:
df.head(6)

,description,number_of_reviews,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,security_deposit,amenities,house_rules,host_since,host_is_superhost,cancellation_policy,neighbourhood_cleansed,reviews_per_month,latitude,longitude,bed_type,ppl_bath,guest_prop,bnb_yield,tier,tier_price,DateDiffHostSince,Location,dist_transit
0,Step into our artistic spacious apartment and ...,168.0,Apartment,Private room,3.0,1.0,1.0,2.0,119.0,2.0,39.0,2.0,0.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",no-smoking/please take off your shoes: cleanin...,2008-09-06,f,strict_14_with_grace_period,Harlem,1.43,40.799205,-73.953676,Real Bed,0.5,0.666667,17425.408,2nd,3rd,3601.0,"(40.79920479936168, -73.95367574543542)",0.115468
1,Renovated apt home in elevator building. Spaci...,7.0,Apartment,Private room,4.0,1.0,2.0,3.0,104.0,1.0,25.0,1.0,100.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",-The security and comfort of all our guests is...,2008-09-07,t,moderate,Kensington,0.23,40.647486,-73.972370,Real Bed,1.0,0.250000,2449.408,3rd,3rd,3600.0,"(40.64748608166989, -73.97236954007957)",0.392200
2,"Find your romantic getaway to this beautiful, ...",32.0,Apartment,Entire home/apt,2.0,1.0,0.0,1.0,325.0,2.0,0.0,1.0,350.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...","Make yourself at home, respect the space and t...",2008-09-09,f,strict_14_with_grace_period,Midtown,0.31,40.753621,-73.983774,Real Bed,0.5,1.000000,10316.800,3rd,1st,3598.0,"(40.753620726572464, -73.98377381114605)",0.085077
3,"This is a spacious, clean, furnished master be...",32.0,Apartment,Private room,2.0,1.0,1.0,1.0,195.0,2.0,50.0,5.0,150.0,"{TV,Internet,Wifi,""Wheelchair accessible"",Kitc...",Guests should be respectful of the home and th...,2008-11-09,f,strict_14_with_grace_period,Williamsburg,0.31,40.708558,-73.942362,Real Bed,0.5,1.000000,6190.080,3rd,2nd,3537.0,"(40.70855778333159, -73.94236227640226)",0.216335
4,Urban retreat: enjoy 500 s.f. floor in 1899 br...,198.0,Other,Entire home/apt,3.0,1.0,1.0,3.0,89.0,1.0,0.0,1.0,500.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Smoking - outside please; pets allowed but ple...,2008-12-07,f,moderate,Clinton Hill,4.42,40.685138,-73.959757,Real Bed,1.0,0.333333,40282.112,2nd,3rd,3509.0,"(40.68513770564301, -73.95975749881094)",0.403653
5,"My place is close to Katz's Delicatessen, Stat...",3.0,Apartment,Entire home/apt,2.0,1.0,2.0,2.0,105.0,2.0,20.0,1.0,150.0,"{Internet,Wifi,Kitchen,Breakfast,Heating,""Smok...",- Please take off your shoes at the door and k...,2008-12-28,f,moderate,East Village,1.20,40.720665,-73.979843,Real Bed,0.5,1.000000,12902.400,2nd,3rd,3488.0,"(40.72066466596736, -73.9798430342775)",0.667301


In [5]:
df.reset_index(drop=True,inplace=True)

In [6]:
df.index

RangeIndex(start=0, stop=37260, step=1)

### Coding Amenities 

#### the amenities feature provides the descriptions of amanities offered to the guests.
As converting the entire amenities list to dummies would create a very large number of features, the unique amenities will only be selected 

df_model_n is the dataframe selected for neural networks and will have both neighbourhood and amenities variabes

df_model will only have a smaller set of categorical variables

In [7]:
df.select_dtypes('object').columns

Index(['description', 'property_type', 'room_type', 'amenities', 'house_rules',
       'host_is_superhost', 'cancellation_policy', 'neighbourhood_cleansed',
       'bed_type', 'tier', 'tier_price', 'Location'],
      dtype='object')

In [8]:

df_model_n=df.drop(['description', 'house_rules', 'host_since',
                   'reviews_per_month','latitude', 'longitude','tier', 'tier_price',
                  'Location'], axis=1)

In [9]:
# obtain a list of all the categorical variable 
dummy_n = list(df_model_n.select_dtypes(exclude=np.number))


In [10]:
dummy_n

['property_type',
 'room_type',
 'amenities',
 'host_is_superhost',
 'cancellation_policy',
 'neighbourhood_cleansed',
 'bed_type']

In [11]:
# Attrib: https://www.kaggle.com/rudymizrahi/how-i-ranked-5th-160-in-deloitte-s-ml-competition
# create a matrix of unique amanities 
df_model_n['amenities'] = df_model_n['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))

np.concatenate(df_model_n['amenities'].map(lambda amns: amns.split("|")).values)

amenities = np.unique(np.concatenate(df_model_n['amenities'].map(lambda amns: amns.split("|")).values))
amenities_matrix = np.array([df_model_n['amenities'].map(lambda amns: amn in amns).values for amn in amenities])

amenities_matrix

df_model_n['amenities'].map(lambda amns: amns.split("|")).head()

np.unique(np.concatenate(df_model_n['amenities'].map(lambda amns: amns.split("|"))))[1:]

amenities = np.unique(np.concatenate(df_model_n['amenities'].map(lambda amns: amns.split("|"))))[1:]
amenity_arr = np.array([df_model_n['amenities'].map(lambda amns: amn in amns) for amn in amenities])
amenity_arr

df_model_n.dtypes

df_model_n = pd.concat([df_model_n, pd.DataFrame(data=amenity_arr.T, columns=amenities)], axis=1)

df_model_n.drop('amenities',axis=1,inplace=True)
df_model_n.head(5)

,number_of_reviews,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,security_deposit,host_is_superhost,cancellation_policy,neighbourhood_cleansed,bed_type,ppl_bath,guest_prop,bnb_yield,DateDiffHostSince,dist_transit,toilet,24-hour check-in,Accessible-height bed,Accessible-height toilet,Air conditioning,Air purifier,BBQ grill,Baby bath,Baby monitor,Babysitter recommendations,Bathtub,Bathtub with bath chair,Beach essentials,Beachfront,Bed linens,Breakfast,Building staff,Buzzer/wireless intercom,Cable TV,Carbon monoxide detector,Cat(s),Changing table,Children’s books and toys,Children’s dinnerware,Cleaning before checkout,Coffee maker,Cooking basics,Crib,Disabled parking spot,Dishes and silverware,Dishwasher,Dog(s),Doorman,Dryer,EV charger,Electric profiling bed,Elevator,Essentials,Ethernet connection,Extra pillows and blankets,Family/kid friendly,Fire extinguisher,Fireplace guards,Firm mattress,First aid kit,Fixed grab bars for shower,Fixed grab bars for toilet,Flat path to front door,Free parking on premises,Free street parking,Game console,Garden or backyard,Ground floor access,Gym,Hair dryer,Handheld shower head,Hangers,Heating,High chair,Host greets you,Hot tub,Hot water,Hot water kettle,Indoor fireplace,Internet,Iron,Keypad,Kitchen,Lake access,Laptop friendly workspace,Lock on bedroom door,Lockbox,Long term stays allowed,Luggage dropoff allowed,Microwave,Mobile hoist,Other,Other pet(s),Outlet covers,Oven,Pack ’n Play/travel crib,Paid parking off premises,Paid parking on premises,Patio or balcony,Pets allowed,Pets live on this property,Pocket wifi,Pool,Pool with pool hoist,Private bathroom,Private entrance,Private living room,Refrigerator,Roll-in shower,Room-darkening shades,Safety card,Self check-in,Shampoo,Shower chair,Single level home,Ski in/Ski out,Smart lock,Smoke detector,Smoking allowed,Stair gates,Step-free access,Stove,Suitable for events,TV,Table corner guards,Washer,Washer / Dryer,Waterfront,Well-lit path to entrance,Wheelchair accessible,Wide clearance to bed,Wide clearance to shower,Wide doorway,Wide entryway,Wide hallway clearance,Wifi,Window guards,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
0,168.0,Apartment,Private room,3.0,1.0,1.0,2.0,119.0,2.0,39.0,2.0,0.0,f,strict_14_with_grace_period,Harlem,Real Bed,0.5,0.666667,17425.408,3601.0,0.115468,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,True,False,True,False,True,False,False,True,True,False,True,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False
1,7.0,Apartment,Private room,4.0,1.0,2.0,3.0,104.0,1.0,25.0,1.0,100.0,t,moderate,Kensington,Real Bed,1.0,0.250000,2449.408,3600.0,0.392200,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,True,False,False,True,False,False,False,True,True,False,False,False,False,True,False,True,True,False,True,False,True,False,False,True,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,True,True,False,True,False,False,False,True,False,False,False,False,False,True,False,True,True
2,32.0,Apartment,Entire 

In [12]:
df_model_n.to_pickle('./data/NYC_Amenities.pkl')

### Get Dummies

#### Neighbourhood has 215 unique features who are candidates for dummying

In [ ]:
df.neighbourhood_cleansed.nunique()

In [14]:
#select all categorical variables for dummying 
dummy = list(df_model_n.select_dtypes(include= object))
dummy

['property_type',
 'room_type',
 'host_is_superhost',
 'cancellation_policy',
 'neighbourhood_cleansed',
 'bed_type']

In [ ]:

df_model_n = pd.get_dummies(df_model_n, columns=dummy,
 drop_first=False)
print("Dataset has {} rows, {} columns.".format(*df_model_n.shape))

In [ ]:
df_model_n.to_pickle('airbnbAmenities_coded.pkl')

####  non- neighbourhood dummies for simple regression

In [ ]:
df.columns

In [ ]:
df_model = df.drop(['description','house_rules', 
                   'reviews_per_month','latitude', 'longitude','tier', 'tier_price',
                  'Location','neighbourhood_cleansed','amenities'], axis=1)

In [ ]:
dummy_d = list(df_model.select_dtypes(include= object))

In [ ]:
dummy_d

In [ ]:
df_model = pd.get_dummies(df_model, columns=dummy_d,
 drop_first=False)
print("Dataset has {} rows, {} columns.".format(*df_model.shape))

In [ ]:
# quick check on the dummied variables 
df_model.info()

In [ ]:
df_model.to_pickle('NYCdum.pkl')